In [ ]:
import warnings
warnings.filterwarnings('ignore')

import yaml
import uuid
import rasterio
import numpy as np
import pandas as pd
from pathlib import Path


In [25]:
# ініціалізувати шляхи
path_to_tiff = './odc/GTOPO_30AS/data/gt30w180n90.tif'
path_save_dataset = './odc/GTOPO_30AS/metadata'
path_product_definition = './odc/GTOPO_30AS/metadata/product_definition/GTOPO_30AS_product_definition.yaml' 


In [10]:
#TODO Додати функцію read_yaml, за допомогою якої можна зчитати yaml файл із метаданими продукта


In [11]:
def get_geo_info_from_tiff(path_to_file):
    # TODO - написати документацію (потрібно описати, яку інформацію можна отримати використовуючи метод - get_transform)
    
    with rasterio.open(path_to_file, 'r') as raster:
        xmin, pixel_width, r_rotation, ymax, c_rotation, pixel_height = raster.get_transform() 
        
    return raster.crs.to_string().lower(), [raster.height,  raster.width],  [pixel_width, r_rotation, xmin, c_rotation, pixel_height, ymax, 0, 0, 1]
    

In [23]:
def create_eo3_dataset(path_product_definition, path_to_tiff, path_save_dataset, datetime=None, file_format="GeoTIFF"):
    """
    Приклад функції для створення метаданих датасету у форматі eo3
    """
    path_to_tiff = Path(path_to_tiff)
    path_save_dataset = Path(path_save_dataset)
    path_product_definition = Path(path_product_definition)
    
    if not datetime:
        datetime = pd.Timestamp('1996-12-01T00:00:00.000000000').strftime("%Y-%m-%d %H:%M:%S.0000000Z")

    product_info = read_yaml(path_product_definition)
    product_name, var_name = product_info['name'], product_info['measurements'][0]['name']
    
    crs, shape, gt = get_geo_info_from_tiff(path_to_tiff)
    
    eo3_doc = {
        
        "id": str(uuid.uuid4()),
        "$schema": 'https://schemas.opendatacube.org/dataset',
        "product": {"name": product_info['name']},
        "crs": crs,
        "grids":{"default":{ "shape": shape, "transform": gt }},
        "measurements": {var_name: {'path': str(path_to_tiff)}},
        "properties": {"datetime": datetime, "odc:file_format": file_format},
        "lineage": {}
            }
    
    metadata_file = path_save_dataset / "{}_{}.yaml".format(product_info['name'], path_to_tiff.stem)
    
    with open(str(metadata_file), 'a') as destination_file_opened:
        yaml.dump(eo3_doc, destination_file_opened, explicit_start=True, sort_keys=False, default_flow_style=None)
        
    return metadata_file

In [31]:
create_eo3_dataset(path_product_definition, path_to_tiff, path_save_dataset)

WindowsPath('D:/odc/GTOPO_30AS/metadata/GTOPO_30AS_gt30w180n90.yaml')

In [ ]:
#TODO Використовуючи наведений код необхідно створити файл із метаданими для всіх 33 тайлів. 

# Рекомендовано виконати це завдання, написавши додаткові функції або оптимізувавши вже існуючий код